# Фильтр Калмана: тайна параметров

In [27]:
import numpy as np

import matplotlib.pyplot as plt
import scipy.interpolate as spint
from tqdm import tqdm
%matplotlib inline

Мы раньше предполагали, что параметры модели в фильтре Калмана известны. То есть, в модели

$$
\begin{align*}
&Y_{t+1} = A Y_t + U_t,\\
&X_{t} = B Y_{t} + W_t,\\
\end{align*}
$$

мы знали $A,B$, ковариационные матрицы шумов $R_x,R_y$ и стартовые параметры $Y_1 \sim N(\xi, \Lambda)$. 

Во многих задачах это почти так. Если вы проектируете робота или (ааа)автомобиль, то вы уже знаете, какие параметры вы наблюдаете с помощью датчиков и как именно. Более того, на приборах указана погрешность, как абсолютная, так и (если есть), относительная. В финансовых задачах вы точно знаете, какие цены вы наблюдаете, но не уверены, что их может определять, поэтому матрица $B$ будет как минимум диагональная. В совокупности такие сведения нам сразу дают матрицы $B,R_x$ и нам неизвестны только параметры динамики.

Есть скрытые переменные, есть скрытая динамика с неизвестными параметрами -- значит, время для ЕМ-алгоритма.

## Строим ЕМ

Как мы видели из лекций, можно получить формулы для М-шага при данных $X_1,..,X_n$:

$$
формулы...
$$

Но в этих формулах есть то, что мы вычислить с ходу не можем. Но может алгоритм фильтрации Калмана! Для того, чтобы вычислить все эти матожидания, нужно
1. Провести процедуру фильтрации (проход вперёд по времени), сохранив нужные логи,
2. Провести процедуру сглаживания (проход назад по времени), сохранив необходимые величины.

А далее дело только в том, чтобы записать аккуратно формулы, потому что дебажить их очень тяжело.

### Фильтрация

Фильтрация  -- это проход вперёд, в ходе которого вычисляются условные матожидания $Y_t$ и $Y_t Y_t^T$ при условии $X_1,..,X_t$

$$
\begin{aligned}
&\tilde{Y}_t = A \hat{Y}_{t-1},\\
&\tilde{P}_t = A P_{t-1}A^T + R_y,\\
&K_t = \tilde{P}_t B^T (B\tilde{P}_tB^T + R_x)^{-1},\\
&\hat{Y}_t = \tilde{Y}_t + K_t(X_t - B \tilde{Y}_t)\\
&P_t = \tilde{P}_t - K_tB\tilde{P}_t.
\end{aligned}
$$

Конкретнее,

$$
\mathbb{E} \left[ Y_t~\vert~ X_1,..,X_t\right] = \hat{Y}_t, \quad \mathbb{E} \left[ Y_tY_t^T ~\vert~ X_1,..,X_t\right] = P_t + \hat{Y}_t\hat{Y}_t^T.
$$


### Сглаживание

Для того, чтобы учесть все $X$ для прогнозов всех $Y$, то есть, посчитать условные матожидания при условии всех $X_1,..,X_n$, есть процедура сглаживания. Но в отличие от случая, когда нам не нужно оценивать параметры, для ЕМ-алгоритма дополнительно  нужно посчитать $\mathbb{E}[Y_iY_{i-1}^T \vert X]$, для этого в конце добавилась новая итерация.

На старте задаётся 

$$
\overline{Y}_n = \hat{Y}_n, ~~ \overline{P}_n = P_n,
$$

а также

$$
V_{n,n-1} = (I - K_nB)A P_{n-1},
$$

эта переменная будет нам помогать оценивать ковариацию $\mathbb{E}[Y_tY_{t-1}^T \vert X] = V_{t,t-1} + \overline{Y}_t\overline{Y}_{t-1}^T$.

Далее следуем для $t=n-1,...,1$ шагами


$$
\begin{aligned}
&S_t = P_t A^T (\widetilde{P}_{t+1})^{-1},\\
&\overline{Y}_t = \hat{Y}_t + S_t(\overline{Y}_{t+1}-\widetilde{Y}_{t+1}),\\
&\overline{P}_t = P_t  + S_t(\overline{P}_{t+1} - \widetilde{P}_{t+1})S_t^T.
\end{aligned}
$$

если $t<n-1$, то дополнительно делаем

$$
V_{t+1,t} = P_{t+1} S_t^T + S_{t+1}(V_{t+2,t+1} - AP_{t+1})S_t^T.
$$

С чертой -- сглаженные значения и их ковариационные матрицы. Отсюда получаем искомые значения:

$$
\mathbb{E} \left[ Y_t~\vert~ X_1,..,X_n\right] = \overline{Y}_t, \quad \mathbb{E} \left[ Y_tY_t^T ~\vert~ X_1,..,X_n\right] = \overline{P}_t + \overline{Y}_t\overline{Y}_t^T.
$$

И не забываем про ковариацию

$$
\mathbb{E}[Y_tY_{t-1}^T \vert X] = V_{t,t-1} + \overline{Y}_t\overline{Y}_{t-1}^T.
$$

## Модель системы и наблюдений

Для экспериментов будем использовать тот же генератор.

In [28]:
from kalmanlib import LinearGaussianSignalGenerator, KalmanFilter

Проверим старый тест, чтобы убедиться, что ничего не сломалось.

In [29]:
A = np.eye(2)
B = np.array([[1.9999,0.31],[-0.2,1.911]])
Ry = np.array([[2,0.00002],[0.00002,3]])*10
Rx = np.array([[1.3,0.0002],[0.0002,3]])*10
T=100

sigGen = LinearGaussianSignalGenerator(A, B, Ry, Rx)

startMean=np.array([10,5])
startCov=np.eye(2)*0.1

In [30]:
start = np.random.multivariate_normal(mean=startMean,cov=startCov)#np.array([10,5])
ys, xs = sigGen.generate(T,start)

In [31]:
filter = KalmanFilter(A, B, Ry, Rx,startMean,startCov)
sig,errs,aprSig,aprErrs = filter.filterSignal(xs)
smoothedSig, smoothedErrs = filter.smoothSignal(sig,errs,aprSig,aprErrs)

In [ ]:
f,(ax1,ax2) = plt.subplots(1,2,figsize=(11,5))

ax1.grid()
ax1.set_title("Показатель номер 1", fontsize=16)
ax1.set_xlabel("t", fontsize=14)
ax1.plot(np.arange(T), ys[0,:])
ax1.plot(np.arange(T), xs[0,:])
ax1.plot(np.arange(T), sig[0,:])
ax1.plot(np.arange(T), smoothedSig[0,:])
ax1.fill_between(np.arange(T),smoothedSig[0,:]-1.96*np.sqrt(smoothedErrs[0,0,:]), smoothedSig[0,:]+1.96*np.sqrt(smoothedErrs[0,0,:]), alpha=0.4 )
ax1.fill_between(np.arange(T),sig[0,:]-1.96*np.sqrt(errs[0,0,:]), sig[0,:]+1.96*np.sqrt(errs[0,0,:]), alpha=0.4 )


ax2.grid()
ax2.set_title("Показатель номер 2", fontsize=16)
ax2.set_xlabel("t", fontsize=14)
ax2.plot(np.arange(T), ys[1,:])
ax2.plot(np.arange(T), xs[1,:])
ax2.plot(np.arange(T), sig[1,:])
ax2.plot(np.arange(T), smoothedSig[1,:])
ax2.fill_between(np.arange(T),smoothedSig[1,:]-1.96*np.sqrt(smoothedErrs[1,1,:]), smoothedSig[1,:]+1.96*np.sqrt(smoothedErrs[1,1,:]), alpha=0.4 )
ax2.fill_between(np.arange(T),sig[1,:]-1.96*np.sqrt(errs[1,1,:]), sig[1,:]+1.96*np.sqrt(errs[1,1,:]), alpha=0.4 )

ax2.legend(["Сигнал(Y)","Измерения(X)","Отфильтрованный сигнал","Сглаженнный сигнал"])
# plt.savefig('filtered.pdf')
plt.show()

Как всегда, выглядит отлично!

### Тестируем ЕМ на этом примере

А как всё будет работать, если попробовать ЕМ-алгоритм? Предположим, что мы НИЧЕГО не знаем априори и всё хотим выучить.

In [33]:
A = np.eye(2)
B = np.array([[1.9999,0.31],[-0.2,1.911]])
Ry = np.array([[2,0.00002],[0.00002,3]])*10
Rx = np.array([[1.3,0.0002],[0.0002,3]])*10
T=100

sigGen = LinearGaussianSignalGenerator(A, B, Ry, Rx)

startMean=np.array([10,5])
startCov=np.eye(2)*0.1

start = np.random.multivariate_normal(mean=startMean,cov=startCov)#np.array([10,5])
ys, xs = sigGen.generate(T,start)

In [ ]:
startA = np.eye(2)
startB = np.eye(2)
startRy = np.eye(2)
startRx = np.eye(2)
startStartMean = np.ones([2])
startStartCov = np.eye(2)
filter = KalmanFilter(A=startA, B=startB, Ry=startRy, Rx=startRx, startMean=startStartMean,startCov=startStartCov)
filter.fit(xs, Niter=200)

In [35]:
sig,errs,aprSig,aprErrs = filter.filterSignal(xs)
smoothedSig, smoothedErrs = filter.smoothSignal(sig,errs,aprSig,aprErrs)

In [ ]:
f,(ax1,ax2) = plt.subplots(1,2,figsize=(11,5))

ax1.grid()
ax1.set_title("Показатель номер 1", fontsize=16)
ax1.set_xlabel("t", fontsize=14)
ax1.plot(np.arange(T), ys[0,:])
ax1.plot(np.arange(T), xs[0,:])
ax1.plot(np.arange(T), sig[0,:])
ax1.plot(np.arange(T), smoothedSig[0,:])
ax1.fill_between(np.arange(T),smoothedSig[0,:]-1.96*np.sqrt(smoothedErrs[0,0,:]), smoothedSig[0,:]+1.96*np.sqrt(smoothedErrs[0,0,:]), alpha=0.4 )
ax1.fill_between(np.arange(T),sig[0,:]-1.96*np.sqrt(errs[0,0,:]), sig[0,:]+1.96*np.sqrt(errs[0,0,:]), alpha=0.4 )


ax2.grid()
ax2.set_title("Показатель номер 2", fontsize=16)
ax2.set_xlabel("t", fontsize=14)
ax2.plot(np.arange(T), ys[1,:])
ax2.plot(np.arange(T), xs[1,:])
ax2.plot(np.arange(T), sig[1,:])
ax2.plot(np.arange(T), smoothedSig[1,:])
ax2.fill_between(np.arange(T),smoothedSig[1,:]-1.96*np.sqrt(smoothedErrs[1,1,:]), smoothedSig[1,:]+1.96*np.sqrt(smoothedErrs[1,1,:]), alpha=0.4 )
ax2.fill_between(np.arange(T),sig[1,:]-1.96*np.sqrt(errs[1,1,:]), sig[1,:]+1.96*np.sqrt(errs[1,1,:]), alpha=0.4 )

ax2.legend(["Сигнал(Y)","Измерения(X)","Отфильтрованный сигнал","Сглаженнный сигнал"])
# plt.savefig('filtered.pdf')
plt.show()

Если позапускать ещё, становится понятно, что дело тут не в количестве итераций и не в точке старта (хотя если задать истинные параметры стартовыми, всё отлично). Алгоритм не ломается, проблем с обращением матриц нет, при этом мы видим, что одно из измерений почти попало в истинный сигнал.

Такой алгоритм не сойдётся к одному значению, потому что с точки зрения правдоподобия есть несколько одинаковых вариантов параметров, которые дадут ту же последовательность $X$. Например, если $B$ умножить на $5$, а $A$ поделить на 5 и скорректировать ковариацонные матрицы ошибок, умножив их на подходящую константу, результат в $X$ не поменяется. Поэтому часть параметров нам придётся из задачи как-то узнать и зафиксировать, чтобы фильтр оценился. Обычно $B$ дана изначально, давайте её зафиксируем.

In [ ]:
filter = KalmanFilter(A=startA, B=B, Ry=startRy, Rx=startRx, startMean=startStartMean,startCov=startStartCov)
filter.fit(xs, Niter=20, fixB=True)

In [38]:
sig,errs,aprSig,aprErrs = filter.filterSignal(xs)
smoothedSig, smoothedErrs = filter.smoothSignal(sig,errs,aprSig,aprErrs)

In [ ]:
f,(ax1,ax2) = plt.subplots(1,2,figsize=(11,5))

ax1.grid()
ax1.set_title("Показатель номер 1", fontsize=16)
ax1.set_xlabel("t", fontsize=14)
ax1.plot(np.arange(T), ys[0,:])
ax1.plot(np.arange(T), xs[0,:])
ax1.plot(np.arange(T), sig[0,:])
ax1.plot(np.arange(T), smoothedSig[0,:])
ax1.fill_between(np.arange(T),smoothedSig[0,:]-1.96*np.sqrt(smoothedErrs[0,0,:]), smoothedSig[0,:]+1.96*np.sqrt(smoothedErrs[0,0,:]), alpha=0.4 )
ax1.fill_between(np.arange(T),sig[0,:]-1.96*np.sqrt(errs[0,0,:]), sig[0,:]+1.96*np.sqrt(errs[0,0,:]), alpha=0.4 )


ax2.grid()
ax2.set_title("Показатель номер 2", fontsize=16)
ax2.set_xlabel("t", fontsize=14)
ax2.plot(np.arange(T), ys[1,:])
ax2.plot(np.arange(T), xs[1,:])
ax2.plot(np.arange(T), sig[1,:])
ax2.plot(np.arange(T), smoothedSig[1,:])
ax2.fill_between(np.arange(T),smoothedSig[1,:]-1.96*np.sqrt(smoothedErrs[1,1,:]), smoothedSig[1,:]+1.96*np.sqrt(smoothedErrs[1,1,:]), alpha=0.4 )
ax2.fill_between(np.arange(T),sig[1,:]-1.96*np.sqrt(errs[1,1,:]), sig[1,:]+1.96*np.sqrt(errs[1,1,:]), alpha=0.4 )

ax2.legend(["Сигнал(Y)","Измерения(X)","Отфильтрованный сигнал","Сглаженнный сигнал"])
# plt.savefig('filtered.pdf')
plt.show()

Прекрасно! Причём сходится за десяток итераций.

## Оценка скорости движения и погрешности

Рассмотрим равномерно и прямолинейно движущийся велосипед.

Модель мира выглядит так:

$$
Y_{t+1} = A Y_t + U_t, ~A = \begin{bmatrix}1 & \Delta t\\ 0 & 1 \end{bmatrix}.
$$

Мир описывается положением велосипеда (первая координата) и скоростью (вторая координата).

Наблюдаем мы только координату (по условному одномерному GPS):

$X_{t+1} = B Y_{t+1} + V_{t+1}, ~ B = [1,0] .$

Для определённости возьмём единичные матрицы в качестве дисперсий шумов, а стартовое состояние зададим с большей дисперсией.

Что известно точно?

* Матрица $A$, если мы знаем с каким $\Delta t$ производятся измерения
* Матрица $B$, если мы хорошо изучили инструкцию к прибору
* Можно ещё зафиксировать одну компоненту шума, например, задать сразу $R_x$ единичной. Если этого не сделать, то в измерении будут складываться два разных независимых шума (из наблюдений и из латентного пространства) и точно разделить их нельзя. Это приводит к известной нам проблеме несходимости.


In [40]:
deltaT = 0.1
A = np.eye(2)
A[0,1] = deltaT
A[1,1] = 1

B=np.array([[1,0]])

Ry = np.eye(2)*2
Rx = np.eye(1)*5

T=50

sigGen = LinearGaussianSignalGenerator(A, B, Ry, Rx)

startMean=np.array([0,10])
startCov=np.eye(2)*10

start = np.random.multivariate_normal(mean=startMean,cov=startCov)#np.array([10,5])
ys, xs = sigGen.generate(T,start)

filter = KalmanFilter(A, B, Ry, Rx,startMean,startCov)

In [ ]:
startA = A
startB = B
startRy = np.eye(2)
startRx = Rx
startStartMean = np.ones([2])
startStartCov = np.eye(2)
filter = KalmanFilter(A=startA, B=startB, Ry=startRy, Rx=startRx, startMean=startStartMean,startCov=startStartCov)
filter.fit(xs, Niter=50,fixB=True,fixA=True,fixRx=True)

In [42]:
sig,errs,aprSig,aprErrs = filter.filterSignal(xs)
smoothedSig, smoothedErrs = filter.smoothSignal(sig,errs,aprSig,aprErrs)

In [ ]:
f,(ax1,ax2) = plt.subplots(1,2,figsize=(11,4))

ax1.grid()
ax1.set_title("Координата", fontsize=16)
ax1.set_xlabel("t", fontsize=14)
ax1.plot(np.arange(T), ys[0,:])
ax1.plot(np.arange(T), xs[0,:])
ax1.plot(np.arange(T), sig[0,:])
ax1.plot(np.arange(T), smoothedSig[0,:])
ax1.fill_between(np.arange(T),smoothedSig[0,:]-1.96*np.sqrt(smoothedErrs[0,0,:]), smoothedSig[0,:]+1.96*np.sqrt(smoothedErrs[0,0,:]), alpha=0.4 )
ax1.fill_between(np.arange(T),sig[0,:]-1.96*np.sqrt(errs[0,0,:]), sig[0,:]+1.96*np.sqrt(errs[0,0,:]), alpha=0.4 )
ax1.legend(["Сигнал(Y)", "Измерения(X)","Отфильтрованный сигнал","Сглаженнный сигнал"])


ax2.grid()
ax2.set_title("Скорость", fontsize=16)
ax2.set_xlabel("t", fontsize=14)
ax2.plot(np.arange(T), ys[1,:])
#ax2.plot(np.arange(T), xs[1,:])
ax2.plot(np.arange(T), sig[1,:])
ax2.plot(np.arange(T), smoothedSig[1,:])
ax2.fill_between(np.arange(T),smoothedSig[1,:]-1.96*np.sqrt(smoothedErrs[1,1,:]), smoothedSig[1,:]+1.96*np.sqrt(smoothedErrs[1,1,:]), alpha=0.4 )
ax2.fill_between(np.arange(T),sig[1,:]-1.96*np.sqrt(errs[1,1,:]), sig[1,:]+1.96*np.sqrt(errs[1,1,:]), alpha=0.4 )

ax2.legend(["Сигнал(Y)","Отфильтрованный сигнал","Сглаженнный сигнал"])
# plt.savefig('filtered.pdf')
plt.show()

## Полет мяча

Сегодня мы попробуем рассмотреть задачу о движении мяча в ваккууме. По пути нам понадобятся знания из физики (на них дадим ссылки на википедию для расширения кругозора).

[Рассматривать мы будем баллистику (задачу о движении снаряда / мяча).](https://en.wikipedia.org/wiki/Projectile_motion)

Из нее известно, что на координатной плоскости можно описать движение при помощи уравнений:

$$
\begin{aligned}
y &= \frac{g}{2}t^2 + v_{y0} t + y_0 \\
x &= v_{x0} t + x_0
\end{aligned}
$$

где $g$ гравитационная постоянная, $t$ время, $v_{x0}$ и $v_{y0}$ начальные скорости по соответствующим осям. Если мяч бросается с ускорением $v$ под углом $\theta$ к горизонту, то начальные скорости $v_{x0}$ и $v_{y0}$ легко вычислить:

$$
\begin{aligned}
v_{x0} = v \cos{\theta} \\
v_{y0} = v \sin{\theta}
\end{aligned}
$$


Так как у нас нет мяча с датчиками, по которым мы сможем получить нужные данные, то решим проблему симуляцией полета мяча.

Случайные добавки (шум) будут отвечать за зашумленность датчиков (независимо от времени).

Теория нам дает описание непрерывной траектории полета мяча, а датчики передают информацию дискретно (с определенной частотой). По этой причине воспользуемся численным приближением ([методом Рунге-Кутта](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods)) задачи Коши, к которой сводится задача о движении мяча по законам Ньютона.                                                              

In [ ]:
!pip install filterpy

In [45]:
import numpy as np
import matplotlib.pyplot as plt
from filterpy.kalman import KalmanFilter

In [46]:
def rk4(y, x, dx, f):
    """computes 4th order Runge-Kutta for dy/dx.
    y is the initial value for y
    x is the initial value for x
    dx is the difference in x (e.g. the time step)
    f is a callable function (y, x) that you supply to 
      compute dy/dx for the specified values.
    """
    
    k1 = dx * f(y, x)
    k2 = dx * f(y + 0.5*k1, x + 0.5*dx)
    k3 = dx * f(y + 0.5*k2, x + 0.5*dx)
    k4 = dx * f(y + k3, x + dx)
    
    return y + (k1 + 2*k2 + 2*k3 + k4) / 6.

def fx(x,t):
    return fx.vel
    
def fy(y,t):
    return fy.vel - 9.8*t


class BallTrajectory2D(object):
    def __init__(self, x0, y0, velocity, 
                 theta_deg=0., 
                 g=9.8, 
                 noise=[0.0, 0.0]):
        self.x = x0
        self.y = y0
        self.t = 0        
        theta = np.radians(theta_deg)
        fx.vel = np.cos(theta) * velocity
        fy.vel = np.sin(theta) * velocity        
        self.g = g
        self.noise = noise
        
        
    def step(self, dt):
        self.x = rk4(self.x, self.t, dt, fx)
        self.y = rk4(self.y, self.t, dt, fy)
        self.t += dt 
        return (self.x + np.random.standard_normal()*self.noise[0], 
                self.y + np.random.standard_normal()*self.noise[1])


Чтобы создать траекторию, начиная с точки (0, 15) со скоростью 100 м/с и углом 60°, мы бы написали:

```python
traj = BallTrajectory2D(x0=0, y0=15, velocity=100, theta_deg=60)
```
    
Затем вызовем traj.step(t) для дискретного изображения полета мяча.


In [ ]:
def test_ball_vacuum(noise):
    y = 15
    x = 0
    ball = BallTrajectory2D(x0=x, y0=y, 
                            theta_deg=60., velocity=100., 
                            noise=noise)
    t = 0
    dt = 0.25
    _, ax = plt.subplots(figsize=(7,4))
    ax.grid()
    ax.set_xlabel("X, meters (distance)")
    ax.set_ylabel("Y, meters (height)")
    while y >= 0:
        x, y = ball.step(dt)
        t += dt
        if y >= 0:
            ax.scatter(x, y, color='r', marker='.', s=75, alpha=0.5)
         
    
    
# test_ball_vacuum([0, 0]) # plot ideal ball position
test_ball_vacuum([1, 1]) # plot with noise 

### Записываем в виде уравнений динамики фильтра Калмана

Переменные состояния в механике, как правило, это координаты. В нашем случае нам необходимо задать координаты положения мяча и их изменения согласно задаче фильтра Калмана. Напомним, что переход состояний в фильтре Калмана должен вычислять текущее состояние на основе предыдущего. Мы предполагаем, что мяч движется в вакууме, поэтому скорость по оси x постоянна, а ускорение по оси y зависит только от гравитационной постоянной $g$. Мы можем дискретизировать уравнения Ньютона, используя хорошо известный [метод Эйлера](https://en.wikipedia.org/wiki/Euler_method) в терминах $\Delta t$:

$$\begin{aligned}
x_t &=  x_{t-1} + v_{x(t-1)} {\Delta t} \\
v_{xt} &= v_{x(t-1)} \\
y_t &= y_{t-1} + v_{y(t-1)} {\Delta t} \\
v_{yt} &= -g {\Delta t} + v_{y(t-1)} \\
\end{aligned}
$$

> Метод Эйлера интегрирует дифференциальное уравнение пошагово, предполагая, что наклон (производная) остаётся постоянной в момент времени $t$. В этом случае производная от позиции — это скорость. На каждом временном шаге $\Delta t$ мы предполагаем постоянную скорость, вычисляем новую позицию, а затем обновляем скорость для следующего временного шага. Существуют более точные методы, такие как метод Рунге-Кутта, но так как мы обновляем состояние с измерением на каждом шаге, метод Эйлера является достаточно точным.

Это подразумевает, что нам нужно включить ускорение по оси $y$ в фильтр Калмана, но не по оси $x$. Это предполагает следующий набор переменных для описания положения мяча:

$$
\mathbf y = 
\begin{bmatrix}
x & \dot x & y & \dot y & \ddot{y}
\end{bmatrix}^\mathsf T
$$

Так как гравитация является постоянной, мы можем её рассматривать как переменную управления. Другими словами, гравитация — это сила, которая известным образом изменяет поведение системы, и она действует на протяжении всего полета мяча.

Уравнение для прогноза состояния выглядит так: $\mathbf{\tilde y} = \mathbf{Ay} + \mathbf{Cu}$. $\mathbf{Ay}$ — это известная функция описывающая полет мяча. Вектор $\mathbf{u}$ позволяет задать управление в фильтр. Для автомобиля управлением будут, например, степень нажатия на педали газа и тормоза, положение рулевого колеса и так далее. В нашем случае для мяча управлением будет гравитация. Матрица $\mathbf{C}$ моделирует воздействие управления на поведение системы. Для автомобиля $\mathbf{C}$ преобразует воздействия тормоза и газа в изменение скорости, а воздействие рулевого колеса — в изменение положения и направления. Для нашей задачи отслеживания полета мяча она будет вычислять изменение скорости из-за гравитации. Поэтому набор переменных состояния будет выглядеть:

$$
\mathbf y = 
\begin{bmatrix}x & \dot x & y & \dot y 
\end{bmatrix}^\mathsf T
$$

### Описываем переход к новому состоянию мира

Система уравнений описывающих полет мяча по сути задается матрицей $\mathbf A$, которую мы умножаем на предыдущее состояние нашей системы, чтобы получить следующее состояние, или априорное $\tilde{\mathbf y} = \mathbf{A\hat{y}}$.

$$
\begin{aligned}
\bar x &= (1*x) + (\Delta t * v_x) + (0*y) + (0 * v_y) \\
\bar v_x &= (0*x) +  (1*v_x) + (0*y) + (0 * v_y) \\
\bar y &= (0*x) + (0* v_x)         + (1*y) + (\Delta t * v_y)   \\
\bar v_y &= (0*x) +  (0*v_x) + (0*y) + (1*v_y) 
\end{aligned}
$$

Обратите внимание, что ни один из членов не включает $g$, гравитационную постоянную. Гравитация учитывается в этой задаче как управление. В матричной форме это записывается как:

$$
\mathbf A = \begin{bmatrix}
1 & \Delta t & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & \Delta t \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

### Опишем управление в задаче
Управление в нашей задаче - это гравитация. Воздействие гравитации на систему вводится через слагаемое $\mathbf{Сu}$. Так получается описать на сколько изменится $\mathbf{\tilde y}$ из-за гравитации. $\mathbf{Cu}$ содержит $\begin{bmatrix}\Delta x_g & \Delta \dot{x_g} & \Delta y_g & \Delta \dot{y_g}\end{bmatrix}^\mathsf T$.

Если мы посмотрим на дискретизированные уравнения, то увидим, что гравитация влияет только на скорость по оси $y$.

$$\begin{aligned}
x_t &=  x_{t-1} + v_{x(t-1)} {\Delta t} \\
v_{xt} &= vx_{t-1}
\\
y_t &= y_{t-1} + v_{y(t-1)} {\Delta t}\\
v_{yt} &= -g {\Delta t} + v_{y(t-1)} \\
\end{aligned}
$$

Таким образом, мы хотим, чтобы произведение $\mathbf{Сu}$ было равно $\begin{bmatrix}0 & 0 & 0 & -g \Delta t \end{bmatrix}^\mathsf T$. В некотором смысле неважно, как мы определим $\mathbf{С}$ и $\mathbf{u}$, если их произведение даёт этот результат. Например, мы могли бы определить $\mathbf{С}=1$ и $\mathbf{u} = \begin{bmatrix}0 & 0 & 0 & -g \Delta t \end{bmatrix}^\mathsf T$. Но это не совсем соответствует нашим определениям для $\mathbf{С}$ и $\mathbf{u}$, где $\mathbf{u}$ — переменная управления, а $\mathbf{С}$ — её функция. Управление для скорости по оси $y$ — это $-g$.

$$\mathbf{С} = \begin{bmatrix}0&0&0&0 \\ 0&0&0&0 \\0&0&0&0 \\0&0&0&\Delta t\end{bmatrix}, \mathbf{u} = \begin{bmatrix}0\\0\\0\\-g\end{bmatrix}$$

Такое представление избыточно; $\mathbf{u}$ в нашей задаче управление воздействует только на вертикальное движение $y$, что подразумевает:

$$
\mathbf{С} = \begin{bmatrix} 
0 & 0 \\ 
0 & 0 \\ 
0 & 0 \\ 
0 & \Delta t 
\end{bmatrix}, \quad 
\mathbf{u} = \begin{bmatrix} 
0 \\ 
-g 
\end{bmatrix}
$$


или

$$
\mathbf{С} = \begin{bmatrix} 
0 \\ 
0 \\ 
0 \\ 
\Delta t 
\end{bmatrix}, \quad 
\mathbf{u} = \begin{bmatrix} 
-g 
\end{bmatrix}
$$

### Опишем наблюдения (функцию измерений)

В предыдущих разделах мы описали устройство "мира". Теперь давайте разберемся как мы его видим через призму датчиков (выборки / данных).
Мы наблюдаем набор координат $\mathbf x = \mathbf{By}$. Где $B$ матрица перехода к измерениям датчиков от истинных состояний мира (движения мяча).  Мы будем предполагать, что у нас есть датчик, который предоставляет нам положение мяча в координатах (x, y), но не может измерять скорости или ускорения. Следовательно, наша функция должна быть:

$$
\begin{bmatrix}x_x \\ x_y \end{bmatrix}= 
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0
\end{bmatrix} 
\begin{bmatrix}
x \\
\dot x \\
y \\
\dot y \end{bmatrix}$$

где

$$\mathbf B = \begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0
\end{bmatrix}$$

### Матрица искаженний (зашумленности датчиков)

Мы будем полагать, что ошибки независимы по осям $x$ и $y$. В данном случае мы начнём с предположения, что ошибки измерения по осям $x$ и $y$ составляют 0,5 квадратных метра. Следовательно:

$$\mathbf R_x = \begin{bmatrix}0.5&0\\0&0.5\end{bmatrix}$$

### Матрица шума положения мяча в мире

Мы предполагаем, что мяч движется в вакууме, поэтому ничто не должно мешать его полету (в противном случае пришлось бы описывать ветер, например). У нас есть 4 переменные состояния, поэтому нам нужна ковариационная матрица размером $4{\times}4$:

$$\mathbf R_y = \begin{bmatrix}0&0&0&0\\0&0&0&0\\0&0&0&0\\0&0&0&0\end{bmatrix}$$

### Начальные условия

Вспомним, что мы вычислили начальную скорость для $x$ и $y$ с использованием тригонометрии и задали значение $\mathbf y$ следующим образом:

```python
omega = radians(omega)
vx = cos(omega) * v0
vy = sin(omega) * v0

f1.x = np.array([[x, vx, y, vy]]).T
```
    
После выполнения всех шагов мы готовы реализовать наш фильтр и протестировать его. Сначала реализация:

In [50]:
#Matrices
#F = A; H = B; B = C; R = R_x; Q = R_y
def ball_kf(x, y, omega, v0, dt, r=0.5, q=0.):
    kf = KalmanFilter(dim_x=4, dim_z=2, dim_u=1)

    kf.F = np.array([[1., dt, 0., 0.],   # x   = x0 + dx*dt  
                     [0., 1., 0., 0.],   # dx  = dx0
                     [0., 0., 1., dt],   # y   = y0 + dy*dt
                     [0., 0., 0., 1.]])  # dy  = dy0

    kf.H = np.array([[1., 0., 0., 0.],
                     [0., 0., 1., 0.]])
    
    kf.B = np.array([[0., 0., 0., dt]]).T
    kf.R *= r
    kf.Q *= q

    omega = np.radians(omega)
    vx = np.cos(omega) * v0
    vy = np.sin(omega) * v0
    kf.x = np.array([[x, vx, y, vy]]).T
    return kf

Теперь мы протестируем фильтр, сгенерировав измерения для мяча с помощью симуляции мяча.

In [ ]:
def track_ball_vacuum(dt):
    global kf
    x, y = 0., 1.
    theta = 35.  # launch angle
    v0 = 80.
    g = np.array([[-9.8]])  # gravitational constant
    ball = BallTrajectory2D(x0=x, y0=y, theta_deg=theta, velocity=v0, 
                            noise=[.2, .2])
    kf = ball_kf(x, y, theta, v0, dt)

    t = 0
    xs, ys = [], []
    _,ax = plt.subplots(figsize=(6,4))
    ax.grid()
    ax.set_xlabel("X, meters (distance)")
    ax.set_ylabel("Y, meters (height)")
    while kf.x[2] > 0:
        t += dt
        x, y = ball.step(dt)
        z = np.array([[x, y]]).T

        kf.update(z)
        xs.append(kf.x[0])
        ys.append(kf.x[2])    
        kf.predict(u=g)     
        p1 = plt.scatter(x, y, color='r', marker='.', s=75, alpha=0.5)
    p2, = plt.plot(xs, ys, lw=2)
    plt.legend([p2, p1], ['Kalman filter', 'Measurements'],
               scatterpoints=1)
    
track_ball_vacuum(dt=1./10)

Мы видим, что фильтр Калмана достаточно точно отслеживает мяч. Однако, как уже объяснялось, это тривиальный пример, поскольку у нас нет шума процесса. Мы можем предсказывать траектории в вакууме с произвольной точностью; использование фильтра Калмана в этом примере — ненужное усложнение. Метод наименьших квадратов дал бы идентичные результаты.

## Отслеживание мяча в воздухе

До этого в описании мира у нас отсутствовали помехи для движения мяча. Давайте введем атмосферу Земли. На траекторию мяча теперь влияют ветер, сопротивление воздуха и вращение мяча. Мы будем считать, что наш датчик — это камера; код, который мы не будем реализовывать, выполнит обработку изображений для обнаружения положения мяча. Это обычно называется "blob detection" в компьютерном зрении. Однако код обработки изображений не совершенен; в любом кадре возможно либо не обнаружить мяч, либо обнаружить ложные положения мяча. Наконец, мы не будем предполагать, что знаем начальное положение, угол или вращение мяча. Основное упрощение, которое мы делаем здесь — это двумерный мир; мы предполагаем, что мяч всегда движется ортогонально плоскости датчика камеры. В нашем случае камера нам дает только двумерные данные.

### Сопротивление воздуха

Существуют различные подходы к этой задаче. Надёжное решение учитывает такие факторы, как шероховатость поверхности мяча (которая влияет на сопротивление нелинейно в зависимости от скорости), эффект Магнуса (вращение вызывает более высокую скорость относительно воздуха на одной стороне мяча по сравнению с противоположной, поэтому коэффициент сопротивления отличается с разных сторон), эффект подъёмной силы, влажность, плотность воздуха и так далее. В данной задаче для простоты рассматривается влияние сопротивления воздуха на не вращающийся бейсбольный мяч. Описание взято из книги Николаса Джордано и Хисао Накаиниши Computational Physics [1997].

По сути этот раздел нужен только для воспроизведение относительно точной симуляции полета мяча. В реальных приложениях обычно невозможно полностью смоделировать физику реальной системы, и мы ограничиваемся некоторой моделью, которая описывает основные характеристики системы. Самой сложной задачей в этом является настройка фильтра Калмана для точной работы с реальными данными. К сожалению, хорошая работа на тестовой выборке в данном случае не гарантирует высокую точность на практике.

Описание физики движения мяча в воздухе.

Мяч, движущийся в воздухе, сталкивается с сопротивлением ветра. Это создаёт силу, которая изменяет траекторию полёта мяча. В работе Джордано это обозначено как:

$$F_{drag} = -B_2v^2$$

где $B_2$ — это коэффициент, полученный экспериментально, а $v$ — это скорость объекта. $F_{drag}$ можно разложить на компоненты по $x$ и $y$ следующим образом:

$$\begin{aligned}
F_{drag,x} &= -B_2v v_x\\
F_{drag,y} &= -B_2v v_y
\end{aligned}$$

Если $m$ — это масса мяча, мы можем использовать $F=ma$ для вычисления ускорения как:

$$\begin{aligned} 
a_x &= -\frac{B_2}{m}v v_x\\
a_y &= -\frac{B_2}{m}v v_y
\end{aligned}$$

Джордано предоставляет следующую функцию для $\frac{B_2}{m}$, которая учитывает плотность воздуха, поперечное сечение бейсбольного мяча и его шероховатость. Это приближение, основанное на испытаниях в аэродинамической трубе и нескольких упрощающих допущениях. Оно выражено в СИ: скорость измеряется в метрах в секунду, время — в секундах.

$$\frac{B_2}{m} = 0.0039 + \frac{0.0058}{1+\exp{[(v-35)/5]}}$$

Вспомним систему полученную методом Эйлера для траектории мяча в вакууме:

$$\begin{aligned}
x &= v_x \Delta t \\
y &= v_y \Delta t \\
v_x &= v_x \\
v_y &= v_y - 9.8 \Delta t
\end{aligned}
$$

Мы можем включить эту силу (ускорение) в наши уравнения, добавив $accel * \Delta t$ в уравнения обновления скорости. Следует вычесть эту компоненту, так как сопротивление будет уменьшать скорость. Код для этого довольно прост; нам просто нужно разбить силу на компоненты $x$ и $y$.

Ниже приведён код, который вычисляет поведение бейсбольного мяча в воздухе, на уровне моря, при наличии ветра. Начальная скорость 160 км/ч

In [ ]:
def drag_force(velocity):
    """ Returns the force on a baseball due to air drag at
    the specified velocity. Units are SI"""

    return velocity * (0.0039 + 0.0058 / 
            (1. + np.exp((velocity-35.)/5.)))

v = 120.
x, y = 0., 1.
dt = .1
theta = np.radians(35)

def solve(x, y, vel, v_wind, launch_angle):
    xs = []
    ys = []
    v_x = vel*np.cos(launch_angle)
    v_y = vel*np.sin(launch_angle)
    while y >= 0:
        # Euler equations for x and y
        x += v_x*dt
        y += v_y*dt

        # force due to air drag    
        velocity = np.sqrt((v_x-v_wind)**2 + v_y**2)    
        F = drag_force(velocity)

        # euler's equations for vx and vy
        v_x = v_x - F*(v_x-v_wind)*dt
        v_y = v_y - 9.8*dt - F*v_y*dt
        
        xs.append(x)
        ys.append(y)
    
    return xs, ys
        
_, ax = plt.subplots(figsize=(6,4))
ax.grid()
x, y = solve(x=0, y=1, vel=v, v_wind=0, launch_angle=theta)
ax.scatter(x, y, color='blue', label='no wind')

wind = 5
x, y = solve(x=0, y=1, vel=v, v_wind=wind, launch_angle=theta)
ax.scatter(x, y, color='green', marker="v", 
                 label='5kmh wind')
ax.legend(scatterpoints=1)

Далее задача усложнится через нелинейное поведение сопротивления воздуха. В таком случае у задачи не будет аналитического решения и придется траекторию вычислять пошагово. При помощи методов Эйлера и Рунге-Кутта можно усложнить симуляцию.

In [55]:
class BaseballPath:
    def __init__(self, x0, y0, launch_angle_deg, velocity_ms, 
                 noise=(1.0, 1.0)): 
        """ Create 2D baseball path object  
           (x = distance from start point in ground plane, 
            y=height above ground)
        
        x0,y0            initial position
        launch_angle_deg angle ball is travelling respective to 
                         ground plane
        velocity_ms      speeed of ball in meters/second
        noise            amount of noise to add to each position
                         in (x, y)
        """
        
        omega = np.radians(launch_angle_deg)
        self.v_x = velocity_ms * np.cos(omega)
        self.v_y = velocity_ms * np.sin(omega)

        self.x = x0
        self.y = y0
        self.noise = noise


    def drag_force(self, velocity):
        """ Returns the force on a baseball due to air drag at
        the specified velocity. Units are SI
        """
        B_m = 0.0039 + 0.0058 / (1. + np.exp((velocity-35.)/5.))
        return B_m * velocity


    def update(self, dt, vel_wind=0.):
        """ compute the ball position based on the specified time 
        step and wind velocity. Returns (x, y) position tuple.
        """

        # Euler equations for x and y
        self.x += self.v_x*dt
        self.y += self.v_y*dt

        # force due to air drag
        v_x_wind = self.v_x - vel_wind
        v = np.sqrt(v_x_wind**2 + self.v_y**2)
        F = self.drag_force(v)

        # Euler's equations for velocity
        self.v_x = self.v_x - F*v_x_wind*dt
        self.v_y = self.v_y - 9.81*dt - F*self.v_y*dt

        return (self.x + np.random.standard_normal()*self.noise[0], 
                self.y + np.random.standard_normal()*self.noise[1])

Теперь мы можем протестировать фильтр Калмана на данных, созданных этой моделью.

In [ ]:
x, y = 0, 1.

theta = 35. # launch angle
v0 = 50.
dt = 1/10.   # time step
g = np.array([[-9.8]])

plt.figure()
ball = BaseballPath(x0=x, y0=y, launch_angle_deg=theta,
                    velocity_ms=v0, noise=[.3,.3])
f1 = ball_kf(x, y, theta, v0, dt, r=1.)
f2 = ball_kf(x, y, theta, v0, dt, r=10.)
t = 0
xs, ys = [], []
xs2, ys2 = [], []

while f1.x[2] > 0:
    t += dt
    x, y = ball.update(dt)
    z = np.array([[x, y]]).T

    f1.update(z)
    f2.update(z)
    xs.append(f1.x[0])
    ys.append(f1.x[2])
    xs2.append(f2.x[0])
    ys2.append(f2.x[2])    
    f1.predict(u=g) 
    f2.predict(u=g)
    
    p1 = plt.scatter(x, y, color='r', marker='.', s=75, alpha=0.5)
plt.grid()
p2, = plt.plot(xs, ys, lw=2)
p3, = plt.plot(xs2, ys2, lw=4)
plt.legend([p1, p2, p3], 
           ['Measurements', 'Filter(R=0.5)', 'Filter(R=10)'],
           loc='best', scatterpoints=1)

Для двух различных настроек фильтра Калмана получили графики. Измерения изображены красными кружочками, фильтр Калмана с $R=0.5$ — синяя линия, а фильтр Калмана с $R=10$ — оранжевая. Эти значения $R$ выбраны лишь для демонстрации влияния погрешностей датчиков на результат, они не предполагают правильной настройки фильтра.

Фильтры справляются плохо. Сначала оба фильтра хорошо отслеживают траекторию, но с течением времени оба расходятся. Это происходит потому, что модель состояния для сопротивления воздуха нелинейна, а фильтр Калмана предполагает линейность. В нашем случае линейность ускорения, которое очевидно нелинейно меняется со временем, поэтому фильтр Калмана ближе к концу траектории овершутит.

Лучший способ исправить результат — это выполнить фильтрацию с использованием нелинейного фильтра Калмана. Однако, trust I am an enginner метод иногда тоже метод. Наш фильтр Калмана предполагает, что мяч находится в вакууме, и, следовательно, отсутствует шум процесса. Однако, так как мяч находится в воздухе, атмосфера оказывает на него воздействие. Мы можем рассматривать эту силу как шум процесса. Это не совсем строгое допущение; во-первых, эта сила далеко не гауссовская. Во-вторых, мы можем вычислить эту силу, поэтому просто утверждать, что «она случайная», не приведёт к оптимальному решению. Но давайте посмотрим, что произойдёт, если мы пойдём по этому пути.

Следующий код реализует тот же фильтр Калмана, что и раньше, но с ненулевым шумом процесса. Я строю два примера: один с Ry=0.1 и другой с Ry=0.01.

In [ ]:
def plot_ball_with_q(q, r=1., noise=0.3):
    x, y = 0., 1.
    theta = 35. # launch angle
    v0 = 50.
    dt = 1/10.   # time step
    g = np.array([[-9.8]])

    ball = BaseballPath(x0=x, 
                        y0=y, 
                        launch_angle_deg=theta, 
                        velocity_ms=v0, 
                        noise=[noise,noise])
    f1 = ball_kf(x, y, theta, v0, dt, r=r, q=q)
    t = 0
    xs, ys = [], []

    while f1.x[2] > 0:
        t += dt
        x, y = ball.update(dt)
        z = np.array([[x, y]]).T

        f1.update(z)
        xs.append(f1.x[0])
        ys.append(f1.x[2]) 
        f1.predict(u=g) 

        p1 = plt.scatter(x, y, c='r', marker='.', s=75, alpha=0.5)

    p2, = plt.plot(xs, ys, lw=2, color='b')
    plt.legend([p1, p2], ['Measurements', 'Kalman filter'])
    plt.show()

plot_ball_with_q(0.01)
plot_ball_with_q(0.1)

Второй фильтр довольно хорошо отслеживает измерения. Кажется, что есть небольшое запаздывание, но совсем незначительное.

Хороша ли эта техника? Обычно нет, но всё зависит от ситуации. Здесь нелинейность силы, действующей на мяч, довольно постоянна и регулярна. Все зависит от соотношения полезного сигнала и шума. Если в данных больше шума, то такой фильтр будет скорее всего его и отслеживать.

In [ ]:
plot_ball_with_q(0.01, r=3, noise=3.)
plot_ball_with_q(0.1, r=3, noise=3.)